# Multi Agents Collaboration


A Multi-Agent System consists of autonomous agents interacting to achieve a goal.

**Core Characteristics**
- Autonomy – Agents operate independently.
- Cooperation – Agents collaborate for shared objectives.
- Coordination – Agents synchronize tasks to avoid redundancy.

## Configuration

Like in the previous exercise we need to setup LLM and configure keys and endpoints

In [5]:
var Endpoint = "<REPLACE_WITH_AZURE_OPENAI_ENDPOINT>";
var Key = "<REPLACE_WITH_AZURE_OPENAI_API_KEY>";
var ChatModelId = "gpt-4o"; // use any model you deployed

## Semantic Kernel

As mentioned earlier, **Semantik Kernel** works really good in more complex scenarios like multi-agent collaboration and communication, so let's use it  

In [6]:
#r "nuget: Microsoft.SemanticKernel, 1.41.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview"

Installed Packages Microsoft.SemanticKernel, 1.41.0 Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview

In [18]:
using System.Threading;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

In [8]:
var kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(ChatModelId, Endpoint, Key)
        .Build();

### Creating multiple agents

Now let's create different agents using our kernel

First let's create **Program Manager** agent to create a plan for creating app

In [25]:
string ProgamManager = """
                        You are a program manager which will take the requirement and create a plan for creating app. 
                        Program Manager understands the user requirements and form the detail documents with requirements and costing. 
                    """;

ChatCompletionAgent ProgramManagerAgent =
    new()
    {
        Instructions = ProgamManager,
        Name = "ProgramManagerAgent",
        Kernel = kernel
    };

Now we need a **Software Engineer** agent to actually create an app

In [26]:
string SoftwareEngineer = """
                            You are Software Engineer, and your goal is create web app using .NET by taking into consideration all
                            the requirements given by Program Manager. 
                        """;

ChatCompletionAgent SoftwareEngineerAgent =
    new()
    {
        Instructions = SoftwareEngineer,
        Name = "SoftwareEngineerAgent",
        Kernel = kernel
    };

And the last one will be **Project Manager** agent who will review the code and make the final approval

In [28]:
string ProjectManager = """
                            You are manager which will review software engineer code, and make sure all client requirements are completed. 
                            You are the guardian of quality, ensuring the final product meets all specifications and receives the green light for release.
                                Once all client requirements are completed, you can approve the request by just responding "approve"
                        """;

ChatCompletionAgent ProjectManagerAgent =
    new()
    {
        Instructions = ProjectManager,
        Name = "ProjectManagerAgent",
        Kernel = kernel
    };

### Termination Strategy

Before creating a chat between these agents we need some kind of termination strategy.  
So everyone would know when to stop  

Let's create one  
Agents will communicate until project manager approves the solution

In [30]:
#pragma warning disable SKEXP0110
sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}

### Group Chat

We created our agents and we configured termination strategy  
Let's make tham talk  

Also besides termination strategy we can specify the maximum iterations limit  
We need to do it not to spend too many resources.  
If we can't get approval for a long time we better improve our prompts and start over

In [ ]:
#pragma warning disable SKEXP0110
AgentGroupChat chat =
    new(ProgramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent)
    {
        ExecutionSettings =
            new()
            {
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        Agents = [ProjectManagerAgent],
                        MaximumIterations = 10,
                    }
            }
    };

Let's start the conversation

In [ ]:
#pragma warning disable SKEXP0001
// Invoke chat and display messages.
var input = """
            I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.
            """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

We can see that the agents communicate back and forth until the condition is satisfied

### Final Thoughts

We learnt that it's better to dedicate a separate agent to its own task and learnt how we can easily do it in .NET  it using **Semantic Kernel** framework